Soil erosion modelling of eucalyptus plantations across the south of Portugal 
Soil MSc - 2025  SRUC

Daniel Fletcher

This Jupyter Notebook goes with the RUSLE2025.pptx deck

## $R$ - Rainfall and Runoff Factor
Based on https://doi.org/10.1016/S0022-1694(01)00387-0

The equation for R from https://doi.org/10.1016/S0022-1694(01)00387-0 is 

$R=\sum_{m=1}^{12} \frac{1}{N} \sum_{i=1}^{N} 7.05 \times rain_{10}(m,i)-88.92\times days_{10}(m,i)$

where  $rain_{10}(m,i)$ is the monthly rainfall for days with $>=10$mm of rain for month $m$ in year $i$ and,

$days_{10}(m,i)$ is the monthly number of days with rainfall $>=10$mm of rain.

First we are going to write some code to get local rainfall data from the internet

We are going to use the Weather API 'OpenMeteo' to derive our R parameter based on the weather at a lat,long during a reference period.

https://open-meteo.com/en/docs/historical-weather-api

In [ ]:
#First install some packages for getting rainfall data from open-meteo
%pip install openmeteo-requests
%pip install requests-cache retry-requests
%pip install geopandas
%pip install geotiff
#THIS MAY TAKE 30 seconds to run

In [ ]:
##Import modules for getting rainfall data from open meteo
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd
import datetime
import numpy as np

We will use the modules to download rainfall data basded on a start date, end date, latitude, and longitude.

Dont worry about understanding this, just know it will get the rainfall data at daily resolution based on the inputs

In [ ]:
def get_weather(start_date,end_date,lat,long):
    #start_date and end_date should be strings of format YYYY-MM-DD
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    
    end_date_dt=datetime.datetime.strptime(end_date,'%Y-%m-%d') #doesn't work
    if end_date_dt<datetime.datetime.today(): #Its histroic weather
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {"latitude": lat,
            "longitude": long,
            "start_date": start_date,
            "end_date": end_date,
            "daily": "precipitation_sum", #or rain_sum (whats the difference)
            "timezone": "GMT"}
    else: #Future Weather
        url = "https://climate-api.open-meteo.com/v1/climate"
        params = {
            "latitude": lat,
            "longitude": long,
            "start_date": start_date,
            "end_date": end_date,
            "models": "HiRAM_SIT_HR", #Chose the climate model you want here!
            "timezone": "GMT",
            "daily": "precipitation_sum" #or rain_sum 
                }
        
    responses = openmeteo.weather_api(url, params=params) #This downloads the weather data as a json
    #Now we unpack the data so its in a nicer format.
    daily=responses[0].Daily()
    daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy() #extract rainfall data
    daily_data = {"date": pd.date_range( #make dates
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left")}
    daily_data["precipitation_sum"] = daily_precipitation_sum
    return daily_data

We can now call the function to get the daily precipiation data from the location we want and start and end date

In [ ]:
daily_data=get_weather("2024-01-01","2024-12-31",37.212281,-8.119963)
print(daily_data)

Daily_data is a dictionary with keys 'date' and 'precipitation_sum'

In [ ]:
daily_data['date'][0] #Access the first date

In [ ]:
daily_data['precipitation_sum'][10] #Access the 10th day precipitation

Using this data we can work out the R value for a specific location and time period using the formula. 

First we write a function to work out $rain_{10}(m,i)$ and $day_{10}(m,i)$

In [ ]:
def calc_rain_day_10(month_precip):
    #month_precip is a months worth of rainfall data at daily resolution
    greater_10= month_precip>=10
    day_10=np.count_nonzero(greater_10) # number of days in the month with rain greater than 10mm
    rain_10=np.sum(month_precip[greater_10]) #summed rainfall in those days
    return rain_10,day_10


In [ ]:
#Test the function
rain_10,day_10=calc_rain_day_10(daily_data['precipitation_sum'][0:30])
print('The number of days with rainfall greater than 10mm in the first 30 days was {} and those days rainfall summed to {:.1f}'.format(day_10,rain_10))
print('The rainfall in this month looked like:')
print(daily_data['precipitation_sum'][0:30])

We are now ready to calculate the R value for a given start_date,end_date, lat and long:

In [ ]:
def calc_R(start_date,end_date,lat,long):
    #A period of 20–25 yr. is recommended for computing the average R
    #Get data. NEEDS INERNET CONNECTION TO RUN
    daily_data=get_weather(start_date,end_date,lat,long)
    years=list(set(daily_data['date'].year)) #A list of all the years
    months=np.arange(1,13) #a list of all the months
    month_R={}#np.zeros(len(years)) #to save the months R value in
    for im,m in enumerate(months):
        month_indxs=daily_data['date'].month==m #pick out indexes in the month m
        dates_month=daily_data['date'][month_indxs] #the dates in the month m
        precip_month=daily_data['precipitation_sum'][month_indxs] # the precipitation data in this month only
        years_R=[] #to save the months R value in
        for iy,y in enumerate(years):
            year_indxs=dates_month.year==y #pick out indexes in the month and year
            precip_month_year=precip_month[year_indxs] #the precipitaiton in this month and year
            rain_10,day_10=calc_rain_day_10(precip_month_year) #call function to get rain_10 and day_10
            result=7.05*rain_10-88.92*day_10 #use the formula
            if result<=0: #Catch those with negatative values
                years_R.append(0)
            else:
                years_R.append(result)  # a list of EI30 value for month m for each year
        month_R[m]=years_R  #{month:[list of yearly EI30s for this month in each year]}
    return [np.mean(v) for v in month_R.values()]

###IGNORE THIS FUNCTION I WAS TESTING SOMETHING
def calc_R_2(start_date,end_date,lat,long):
    
    #A period of 20–25 yr. is recommended for computing the average R
    #Get data 
    daily_data=get_weather(start_date,end_date,lat,long)
    years=list(set(daily_data['date'].year)) #A list of all the years
    months=np.arange(1,13) #a list of all the months
    year_R={}#np.zeros(len(years)) #to save the years R value in
    for iy,y in enumerate(years):
        year_indxs=daily_data['date'].year==y #pick out indexes in the year y
        dates_year=daily_data['date'][year_indxs] #the dates in the year y
        precip_year=daily_data['precipitation_sum'][year_indxs] # the precipitation data in this year only
        month_R=[] #to save the months R value in
        for im,m in enumerate(months):
            month_indxs=dates_year.month==m #pick out indexes in the month m
            precip_month=precip_year[month_indxs] #the precipitaiton in this month
            rain_10,day_10=calc_rain_day_10(precip_month) #call function to get rain_10 and day_10
            result=7.05*rain_10-88.92*day_10 #use the formula
            if result<=0:  
                continue
            else:
                month_R.append(result)
        year_R[y]=month_R  
    return year_R


We can now call the function over a period to work out the mean monthly-R value for each month over a period

In [ ]:
monthly_mean_Rs=calc_R("1970-01-01","1997-12-31",37.310764, -8.826724) #Aljezur

In [ ]:
print('The yearly R value from 1970-1997 in Aljezur is {} MJmm/ha/h/year'.format(monthly_mean_Rs))

**EXERCISE**: Have a play with changing the reference period and location. Use google maps to get different lat, longs in the Algarve. You can use table 2 in https://doi.org/10.1016/S0022-1694(01)00387-0 to compare your results too.

**It is recommended the reference period is at least 20 years long to get a reliable average** 

In [ ]:
##write your code here. 

##  $K$ - Soil erodibility factor

Based on https://core.ac.uk/download/pdf/6506014.pdf table 7

$K=(0.043\times pH +\frac{0.62}{OM}+0.0082\times Sa - 0.0062\times Cl_{ratio})\times Si_{ratio}\times 0.1317$

In [ ]:
def calc_K(pH,Sa,Cl,Si,OM):
    #pH - pH of soil
    #Sa - percent Sand of soil
    #Cl - percent Clay of soil
    #Si - percent Silt of soil
    #OM - OM percent of soil
    #output units are th/MJ/mm
    Cl_ratio=Cl/(Sa+Si)
    Si_ratio = Si/100
    return (0.043*pH+0.62/OM+0.0082*Sa-0.0062*Cl_ratio)*Si_ratio*0.1317 #convert to SI
    

We can try it out on an example soil: 

In [ ]:
#An example loamy fine sand. Should have a K value of 0.009 according to https://core.ac.uk/download/pdf/6506014.pdf
pH=5.5
OM=2.4
Sa=75
Si=6
Cl=19
K=calc_K(pH,Sa,Cl,Si,OM)
print(K)

**EXERCISE**: Check our equation works for some other example soils of table 7 in https://core.ac.uk/download/pdf/6506014.pdf below. Note the K vlaues in the table are in imperial units, so you will need to convert with a factor of 0.1317 to make it comparable to our equation output

In [ ]:
#Write your code here

We can now calculate the annual tonnage of soil lost a year for our loamy fine sand. Assuming the soil is tilled and at a 9% slope of lenght 22m

In [ ]:
R=K*np.nansum(monthly_mean_Rs)
print('The yearly average loss of soil (1970-1997) for a tilled sandy loam field at 9% slope of 22m in Aljezur is {:.2f} t/ha/year'.format(R))

## $LS$ - slope length and steepness factor

Based on https://doi.org/10.1002/9781118351475.ch22, however, you may choose other equtions from table 5 in https://doi.org/10.1016/S0022-1694(01)00387-0 or beyond, provided the paramaters can be measured

$LS=(\frac{l}{22})^{0.5}(0.065+0.045s+0.0065s^{2})$

where $s$ is the slope steepness in percent and $l$ is the slope length in m

**EXERCISE** write a function called calc_LS to determine LS based on $s$ and $l$.

What should happend to LS when the slope is 9% and 22m long? Test your function



In [ ]:
def calc_LS(s,l):
    #power = **
    return (l/22)**0.5*(0.065+0.045*s+0.0065*s**2)
    

In [ ]:
print(calc_LS(9,22))

**Exercise**: Calculate the annual tonnage of soil lost a year for our loamy fine sand. Assuming the soil is tilled and at a 15% slope of 22m

In [ ]:
#Write code here

## $A$ - Putting it all together

Now we have made funtions for $R$, $K$, $LS$ and have parameters for $C$ and $P$ we can write a function to calculate $A$ t/ha/year of soil loss

In [ ]:
def calc_Amonthly(start_date,end_date,lat,long,pH,Sa,Cl,Si,OM,s,l,C,P):
    monthly_mean_Rs=calc_R(start_date,end_date,lat,long)
    R=np.array(monthly_mean_Rs)
    K=calc_K(pH,Sa,Cl,Si,OM)
    LS=calc_LS(s,l) ###MAKE SURE YOU FINISH THE EXERCISE TO COMPLETE THIS FUNCTION
    return R*K*LS*C*P #t/ha/year/

def calc_A(start_date,end_date,lat,long,pH,Sa,Cl,Si,OM,s,l,C,P):
    monthly_mean_Rs=calc_R(start_date,end_date,lat,long)
    R=np.nansum(monthly_mean_Rs)
    K=calc_K(pH,Sa,Cl,Si,OM)
    LS=calc_LS(s,l) ###MAKE SURE YOU FINISH THE EXERCISE TO COMPLETE THIS FUNCTION
    return R*K*LS*C*P #t/ha/year/

In [ ]:
#Test the function
#R params
start_date='2010-01-01'
end_date='2022-12-31'
lat=37.2
long=-7.516667
#Soil K params - to measure in field
pH=5.5
OM=2.4
Sa=75
Si=6
Cl=19
#Slope LS params - to measure in field
l=22
s=9
##C and P factors
C=0.005 #Forest, is this valid for eucalyptus?!
P=1 #No conservation starts
A=calc_A(start_date,end_date,lat,long,pH,Sa,Cl,Si,OM,s,l,C,P) ##Call the function

In [ ]:
print('Soil loss for this scenario is {} t/ha/year'.format(A))

**Research question 1: Has climate change affected soil erosion from 1940 to present?**

We will answer this by evaluating our erosion function for our reference soil for 1940-1959,1960-1979,....etc (open-meteo only goes back in 1940). We will not consider landuse change.


In [ ]:
import datetime #module to handle dates
import matplotlib.pyplot as plt #moudle to to make plots

starts=[datetime.date(1940+n,1,1) for n in range(0,80,20)] #a list of start dates
ends=[datetime.date(1959+n,12,31) for n in range(0,80,20)] #a list of end dates

erosions=[]
for i in range(len(starts)): #loop through the dates
    A=calc_A(starts[i].strftime("%Y-%m-%d"),ends[i].strftime("%Y-%m-%d"),lat,long,pH,Sa,Cl,Si,OM,s,l,C,P) ##Call the function
    erosions.append(A*1000) #save the result as kg/ha/year. This doesnt consider landuse change!

In [ ]:
##Now we can plot
plt.plot(erosions)
plt.xticks(ticks=[0,1,2,3],labels=['1940-1960','1960-1980','1980-2000','2000-2020'])
plt.ylabel('kgSoil/ha/year')

In [ ]:
## With this code we look at each months erosion

starts=[datetime.date(1940+n,1,1) for n in range(0,80,20)] #a list of start dates
ends=[datetime.date(1959+n,12,31) for n in range(0,80,20)] #a list of end dates

erosions=[]
for i in range(len(starts)): #loop through the dates
    A=calc_Amonthly(starts[i].strftime("%Y-%m-%d"),ends[i].strftime("%Y-%m-%d"),lat,long,pH,Sa,Cl,Si,OM,s,l,C,P) ##Call the function
    erosions.append(A*1000) #save the result as kg/ha/year. This doesnt consider landuse change!

counter=0
for e in erosions:   
    plt.plot(e,label=starts[counter].strftime("%Y-%m-%d"))
    counter+=1
plt.legend()



## Possible Tasks for trip

- Measure Soil and Slope params for a variety of sites during the trip. Look for contasting management and cover.

- Contour maps could give slope params for many sites -> GIS project?

- Make a monthly version of calc_A() which returns each months soil loss.

- Which months have the highest soil loss?

- How has climate change affected soil loss in each month? Open-meteo has a climate change model to look at possible future climates https://open-meteo.com/en/docs/climate-api. Write another version of get_weather() to use this api and look at future climates

- How does felling or forrest fires affect soil loss? 
    Burning: https://www.jswconline.org/content/59/1/36.short, Felling:Table 6 in https://doi.org/10.1016/j.catena.2006.01.006 might be an idea on how feling affects C over time. 




## This section loads up a datae base of historical soil measurements in Portugal  

In [ ]:
%pip install geopandas
%pip install contextily

In [ ]:
###Some packages to open up the soil database: https://projects.iniav.pt/infosolo/
import geopandas as gpd
import pandas as pd
import contextily as cx

In [ ]:
#Load up data as a csv and convert it to a geo-data set
df=pd.read_csv('../data/external/pt_infosolo.csv')
df["geometry"] = gpd.points_from_xy(df.longitude, df.latitude, crs="EPSG:4326")
gdf=gpd.GeoDataFrame(df,geometry='geometry')
gdf['om']=gdf['oc']*1.75 #Approximate organic matter % as 1.75 organic carbon%


##Where was the new sample taken? We use this to look for the closest 
lat=37.323304 
long=-8.834470
#
point_df=pd.DataFrame({'geometry':gpd.points_from_xy(x=[long], y=[lat],crs="EPSG:4326")})
point_gdf=gpd.GeoDataFrame(point_df,geometry='geometry')




In [ ]:
point_data=point_gdf.sjoin_nearest(gdf,how='inner')

In [ ]:
point_data.iloc[0]

In [ ]:
#make it so plots open a new window
%matplotlib qt 
ax=gdf.plot()
point_gdf.plot(ax=ax,color='r')
cx.add_basemap(ax, crs=gdf.crs,source=cx.providers.CartoDB.Voyager)

In [ ]:
###Look at where the Eucalyptos plantations measurements are and their organic 
gdf_euc=gdf[gdf['land_use']=='Eucalypt forest'] #only eucalypt forresy
gdf_euc_top=gdf_euc[gdf_euc['hor_top']==0] #only the samples where it is the top of the soil
ax=gdf_euc_top.plot(column='om',legend=True)
cx.add_basemap(ax, crs=gdf.crs,source=cx.providers.CartoDB.Voyager)


## Code to look at the digital evelation map 

In [ ]:
##For DEM maps
import rasterio
from rasterio.windows import Window
DEM_path='../data/dem_srtm_pt_25m.geotiff.tif'



In [ ]:
def get_elevation(DEM_path,lat1,long1):
    #We need to convert to a the portugese crs
    df=pd.DataFrame({'lat':[lat1],'long':[long1]})
    gdf=gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long, df.lat), crs='EPSG:4326')
    gdf_port=gdf.to_crs('EPSG:3763')
    lat_port=gdf_port.geometry.to_list()[0].x
    long_port=gdf_port.geometry.to_list()[0].y
    with rasterio.open(DEM_path) as DEM_file:
        meta=DEM_file.meta
        rowcol = rasterio.transform.rowcol(meta['transform'], xs=lat_port, ys=long_port)

        y = rowcol[0]
        x = rowcol[1]
        print((x,y))

        # Load specific pixel only using a window
        width=50
        height=50
        window = Window(x-width/2,y-height/2,width,height)
        arr = DEM_file.read(window=window)
    return arr[0],window

In [ ]:
#37.289803, -8.858953
#37.327615, -8.725525
#39.102983, -8.758106
height_arr,window=get_elevation(DEM_path,lat, long)


In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.contour(np.flipud(height_arr),levels=20)
plt.figure()
plt.imshow(height_arr)

In [ ]:
height_arr[25,25] #elevation at your location

In [ ]:
gradient=(height_arr[25,25]-height_arr[25,26])/25

In [ ]:
max_height=np.max(height_arr)
loc_max=np.where(height_arr==max_height)
min_height=np.min(height_arr)
loc_min=np.where(height_arr==min_height)

In [ ]:
print(max_height)
print(loc_max)

In [ ]:
print(min_height)
print(loc_min)

In [ ]:
height_arr[8,37]